<a href="https://colab.research.google.com/github/lunamatic77/testproject/blob/main/Interview_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install stripe

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 11.4 MB/s eta 0:00:00


In [ ]:
import stripe

#This is my test key account under normal circumstances I shouldn't share it with anyone
stripe.api_key = "sk_test_51OtperADP7qnqFiUMBI4s1vN0YiyYN4xpUjMzQAhyZJDlxikPEOtjohIxwDIvy6S2O8qtsh888bSqgZBCSysMtIT00LA23ziwS"

# Question 1 | Saving Customer Objects

Stripe allows you to save credit card details for later. Using an API call, create a customer object leveraging a Visa test payment method. For all of the following questions below, you should continue to charge against the same


In [ ]:

# Creating an customer with one of the test payment methods.
customer = stripe.Customer.create(
  name="Guy Incognito",
  email="guy-incognito@example.com",
  payment_method="pm_card_visa_cartesBancaires" #Test Visa payment method
)

payment_methods=stripe.Customer.list_payment_methods(
  customer.id,
  limit=3,
)

if len(payment_methods.data) > 0:
  payment_method_id =  payment_methods.data[0].id

print(payment_method_id)
print(customer.id)

pm_1OtzaYADP7qnqFiUTdURlM6b
cus_PjSV9mSDzEmeh3


# Question 2 | Auth & Capture

Stripe allows its users to auth and capture. This is most commonly used by retailers (like Amazon) who authorize your credit card at the time of checkout, but only capture the charge after they have confirmed that their warehouse has the item or once they have shipped the item.



**2a.** Use an API call to create an authorization for $100. Provide the Payment Intent ID related to this authorization.

In [ ]:
payment_intent=stripe.PaymentIntent.create(
  amount=10000, #10,000 cents :-)
  currency="usd",
  payment_method=payment_method_id,
  customer=customer.id,
  capture_method='manual', #Since we don't want to capture yet we will set 'manual'
  confirm=True,
  return_url="http://www.stripe.com/"
)

print(payment_intent.id)
print(payment_intent.status)


pi_3Ou0ntADP7qnqFiU0yDQPZIV
requires_capture


2b. Screenshot

**2c.** Use an API call to create a capture for $75. What happens to a charge if you only capture for a portion of an authorization, and not the full amount?

In [ ]:
payment_intent=stripe.PaymentIntent.capture(payment_intent.id,
                                            amount_to_capture=7500
                                            )
print(payment_intent.id)
print(payment_intent.status)

pi_3Ou0ntADP7qnqFiU0yDQPZIV
succeeded


The original reserved amount is refunded and the user is only charged the amount that was captured ($75 USD)

**2d.** How do steps 2a-2c show on the customer’s bank statement?

- When the reserve for $100 USD is created, it should show up on the  partial bank statement that users can see on their bank's website or App, the amount will be showed as "in transit" or "pending".

- On the partial statement from the bank's website or app, when the final amount is captured, the user should see a refund for the amount reserved "+100USD", and a new charge for $75USD which is not pending or "in transit".

- The monthly official final statement should only include the $75USD charge and not the temporary reserved amount.



**2e.** Name and describe 2 distinct business models/use cases that would require auth and then capture at a later time.

Besides the use case for online stores like Amazon which is mentioned in the question, these industries use this model:

- Airlines, in many countries consumer laws allow users to "repent" from a purchase, so when users buy a flight, many airlines reserve and amount for some period, for example 24 or 48 hours, after that period elapses they make a capture.

- Car rental companies create a reserve  for certain amount when they deliver a car to a customer, after the customer returns the car, the rental company may refund 100% of the reserved amount or charge the user if the circumnstances require it.

- Online stores/marketplaces that sell digital goods (apps, vod purchase or rental): some online stores allow the users to "return" a digital good within certain amount of time after the purchase, for example 2 hours. In this case the online store could reserve the amount, and capture it after the period.





# Question 3 | Stripe Connect

Through our Connect platform product, Stripe enables platforms like Lyft to collect payments from riders and also payout drivers. In Stripe API terminology, a rider is a “customer” and a driver is a “connected account.” There are three different types of Connect accounts. For this question, you will be making API calls with Custom Connect accounts. Use this Connect test data as needed.

**3a.** Create a Custom Connect connected account. Provide the account ID.

In [ ]:

test_connect_account=stripe.Account.create(
  type="custom",
  country="US",
  capabilities={
    "card_payments": {"requested": True},
    "transfers": {"requested": True},
  },
  external_account='btok_us',
  tos_acceptance={
    "ip": "172.18.80.19",
    "date": 1547923073,
  }
)

print(test_connect_account.id)


acct_1OtzyEPOVSzKex68


*In real life, the connected account's holder should provide valid information to verify the account.  For this test scenario, we will need to provide test data to have an usefule test connected account *

In [ ]:
# Modifying the test account to provide test data for validation

test_connect_account=stripe.Account.modify(
  test_connect_account.id,
  metadata={"order_id": "6735"},
  business_type="individual",
  business_profile={
      "mcc":5045,
      "url":"https://bestcookieco.com",
  },
  company={
      "address":{
          "city":"Schenectady",
          "line1":"123 State St",
          "postal_code":"12345",
          "state":"NY",
      },
      "tax_id":"000000000",
      "name":"The Best Cookie Co"
  },
  individual={
      "address":{
        "city":"Schenectady",
        "line1":"123 State St",
        "postal_code":"12345",
        "state":"NY",
      },
      "dob":{
          "day":"01",
          "month":"01",
          "year":"1902",
      },
      "email":"mrt@example.com",
      "first_name":"Gunnar",
      "last_name":"Gunnarsson",
      "phone":"8888675309",
      "id_number":"000000000",
      "ssn_last_4":"0000"
  }
)

# We shouldn't see pending requirements
print(test_connect_account.requirements)
print(test_connect_account.id)

{
  "alternatives": [],
  "current_deadline": 1711578358,
  "currently_due": [],
  "disabled_reason": null,
  "errors": [],
  "eventually_due": [],
  "past_due": [],
  "pending_verification": [
    "business_profile.url"
  ]
}
acct_1OtzyEPOVSzKex68


**3b**. Create a “destination” charge for a Lyft ride in which the rider pays 20USD  and the driver receives $15. Provide the Payment Intent ID.

In [ ]:
test_destination_intent=stripe.PaymentIntent.create(
  amount=2000,
  currency="usd",
  payment_method=payment_method_id,
  customer=customer.id,
  #capture_method='manual',
  confirm=True,
  application_fee_amount=500, # Platforms's fee
  transfer_data={"destination":test_connect_account.id},
  return_url="http://www.stripe.com/",
)

print(test_destination_intent.id)
print(test_destination_intent.status)


pi_3Ou0e9ADP7qnqFiU1x59yygi
requires_capture


**3c.** For this ride, how much is Lyft’s platform fee?

$5 USD

In [ ]:
print(test_destination_intent.application_fee_amount)


500


**3d.** How much is the Stripe processing fee for this ride?

2000


**3e.** What are Lyft's net earnings for this ride?

**3f.** Now, try to have Lyft charge the driver $2 to cover the cost of the neon-lighted Lyft sign that sits in their dashboard. Provide the ID for that request.

In [ ]:
# The signup flow of our hypothetical platform's app should include a step
# where the drivers provide payment information, so we could create a payment
# method for this kinds of charges, we could use these API calls to set the payment method






# If the payment method has not been set we can create the PaymentIntent, but the
# status will be "requires_payment_method" and we will need the driver to accomplish
# that step.

direct_charge = stripe.PaymentIntent.create(
  amount=200,
  currency="usd",
  automatic_payment_methods={"enabled": True},
  stripe_account='acct_1OtxTMPMvHR1xzRb',  # We charge the connected account.
  #stripe_account=test_connect_account.id,  # We charge the connected account.
)
print(direct_charge.id)
print(direct_charge.status)

pi_3OuAoUPMvHR1xzRb1xlh5jrN
requires_payment_method


**3g.** Tell us about how you went about solving 3F. What is your API call doing?

From the guide: *"With Connect, you can make charges directly on the connected account and take fees in the process."*

So in this case, I think that creating a payment intent using the id of the connected account.

This requires that the user sets a payment method. In our hypothetical scenario when the apps creates the Connected account, it could also create a payment method. The flow to sign up a new driver, besides providing a form to enter all the information, it could also provide a step to enter a payment method.


**3h.** Aside from rideshare, name a business or industry that should use Stripe Connect and why.

Connects works very well for industries where freelancers provide services, so it could also be used for:
- Online Psychoterapy platforms where independent psychologist can sign up to provide therapy to the users of the platform. The platform could use Stripe Connect accounts to pay the psychologists.



# Question 4 | Responding to Users

For this prompt, imagine you're a TAM supporting a user across Stripe's portfolio of products and they wrote into Stripe:
"Good news, we got go ahead for our planned streaming service. Development will start next week and we’re

> Indented block
looking to understand how we can charge our users on a monthly basis, it looks like Stripe can help with that?
We have two plans in mind:
> *   Plan A: Flat rate of \$24.99 per month for unlimited usage of the service

> *   Plan B: \$10.99 for the first 100 GB and \$1.00 per subsequent 10 GB
Can you please give an overview of how this could be achieved through the API? As you know, we use Python in our environment.
Finally, if we wanted to introduce a coupon into the mix, how would that work?
Best,
>
>[Customer's Name]"

**How would you respond?**


